*To be run on Sage kernel, with eel installed*

In [1]:
from hourglass import App

In [2]:
# retreive example data for constructor
from examples import get_example_2
g = get_example_2()

In [3]:
app = App(graph=g)

In [4]:
"""
Initializes server at http://localhost:8000/draw.html
Open in browser for now (easier to debug)
"""

app.start()

start with (0/17).0.0
switch to (17/0).1.0
unfilled turn to (17/19).0.0
switch to (19/17).2.0
filled turn to (19/24).0.1
switch to (24/19).2.1
unfilled turn to (24/4).1.0
start with (0/17).0.0
switch to (17/0).1.0
unfilled turn to (17/19).0.0
unfilled turn to (17/18).2.0
switch to (18/17).1.0
filled turn to (18/16).2.0
filled turn to (18/23).0.0
switch to (23/18).2.0
unfilled turn to (23/20).1.0
unfilled turn to (23/20).1.1
switch to (20/23).2.1
filled turn to (20/23).2.0
filled turn to (20/29).0.0
switch to (29/20).3.0
unfilled turn to (29/21).2.0
unfilled turn to (29/22).1.0
switch to (22/29).1.0
filled turn to (22/28).2.1
filled turn to (22/28).2.0
switch to (28/22).2.0
unfilled turn to (28/22).2.1
unfilled turn to (28/9).1.0
start with (0/17).0.0
switch to (17/0).1.0
unfilled turn to (17/19).0.0
switch to (19/17).2.0
filled turn to (19/24).0.1
switch to (24/19).2.1
unfilled turn to (24/4).1.0


SystemExit: 

In [9]:
g = app.conv_to_sage()
g.show()

NameError: name 'Graph' is not defined